In [6]:
pip install finnhub-python


Note: you may need to restart the kernel to use updated packages.


In [7]:
import finnhub
import datetime
finnhub_client = finnhub.Client(api_key='c9nclciad3if6tl7i78g')

In [107]:
#This makes sure that the 50 day moving average has been above the 200 for teh past 2 weeks to ensure no false signals 
#are raised
def check_duration(data, days):
    # Check if the 50 day moving average stayed above the 200 day moving average
    # for at least the given number of days
    ma50list = []
    ma200list = []
    i = 0
    for x in data:
        if i >=50:
            ma50list.append(int(sum(data[i-49:i])/50))
        if i >=200:
            ma200list.append(int(sum(data[i-199:i])/200))
        i+=1
    duration = 0
    i = (-13)
    
    while i<0:
        if ma50list[i] > ma200list[i]:
            duration += 1
            if duration >= days:
                return True
            
        i+=1
    return False

In [108]:
def calc_moving_average(data, days):
    # Calculate the moving average for the given number of days
    # using the historical data
    ma = sum(data[-days:]) / days
    return ma

In [109]:
def find_stocks(stocks):
    now = datetime.datetime.now()

    # Get the Unix time stamp for the current date and time
    unix_timestamp_now = now.timestamp()
    # Subtract 363 days from the current date and time
    past_date = now - datetime.timedelta(days=363)
    # Get the Unix time stamp for the date and time that is 363 days before the current date and time
    past_unix_timestamp = past_date.timestamp()
    results = []
    
    for stock in stocks:
        # Retrieve the historical data for the stock
        data = finnhub_client.stock_candles(stock, 'D', int(past_unix_timestamp),int(unix_timestamp_now))
        data = data['c']
       
        # Calculate the 50 day moving average and the 200 day moving average
        ma50 = calc_moving_average(data, 50)
      
        ma200 = calc_moving_average(data, 200)
        
        #Check if the 50 day moving average went above the 200 day moving average in the past 2 weeks
        if ma50 > ma200 and check_duration(data, 12):
            results.append(stock)
    return results
stock_list = ['AAPL', 'MSFT', 'TSLA', 'JNJ', 'JPM', 'XOM', 'V', 'WMT', 'TSM', 
               'NVDA', 'BAC', 'CVX', 'ABBV', 'PFE', 'MRK', 'KO', 'ORCL', 'BABA', 
               'CSCO', 'ABT', 'WFC', 'DIS', 'BMY', 'NKE', 'NEE', 'VZ', 'MS', 'COP', 
               'CMCSA', 'SCHW', 'T', 'C', 'INTC', 'AMD', 'BP', 'BA','JD', 'GE', 'TJX', 
              'MO', 'VALE', 'PBR', 'SLB', 'CSX', 'PBR-A', 'MRNA', 'OXY', 'MU', 'GM', 'UBER', 
              'F', 'FCX', 'ABEV', 'ITUB', 'SU', 'DVN', 'KMI', 'LYG', 'CVE', 'ET', 'CARR', 
              'KR', 'LVS', 'HAL', 'CPNG', 'BBD', 'PCG', 'HPQ', 'BKR', 'GOLD', 'NOK', 'SIRI', 
              'LUV', 'DAL', 'HBAN', 'RF', 'ZM', 'HPE', 'BEKE', 'MRO', 'LI', 'UMC', 'NIO', 'LCID', 
              'PINS', 'SNAP', 'EQT', 'APA', 'PLTR', 'UAL', 'HST', 'SUZ', 'CCL', 'TME', 'CCJ', 'GFI']
r = find_stocks(stock_list)
print(r)

FinnhubAPIException: FinnhubAPIException(status_code: 429): API limit reached. Please try again later. Remaining Limit: 0